In [5]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [6]:
target_df = pd.read_csv('../Get_surrounding_assay_from_ChEMBL/acute_myeloid_leukemia_acceptable_approved_drugs_with_chembl_id.csv')
target_df

,drug_name,resource_url,parent_chembl_id,chembl_id_synonyms,specificity,assay_chembl_id,target_chembl_id
0,Azacitidine,https://www.cancer.gov/about-cancer/treatment/...,CHEMBL1489,"['CHEMBL1489', 'CHEMBL3250420', 'CHEMBL3250421']",acceptable,"['CHEMBL650218', 'CHEMBL635477', 'CHEMBL857778...","['CHEMBL2362975', 'CHEMBL2094118', 'CHEMBL386'..."
1,Cerubidine (Daunorubicin Hydrochloride),https://www.cancer.gov/about-cancer/treatment/...,CHEMBL178,"['CHEMBL178', 'CHEMBL1563', 'CHEMBL1200475']",acceptable,"['CHEMBL756922', 'CHEMBL879955', 'CHEMBL705941...","['CHEMBL614516', 'CHEMBL614514', 'CHEMBL236297..."
2,Cytarabine,https://www.cancer.gov/about-cancer/treatment/...,CHEMBL803,"['CHEMBL803', 'CHEMBL1256472']",acceptable,"['CHEMBL736675', 'CHEMBL743774', 'CHEMBL742542...","['CHEMBL613859', 'CHEMBL2362975', 'CHEMBL61427..."
3,Dexamethasone,https://www.cancer.gov/about-cancer/treatment/...,CHEMBL384467,['CHEMBL384467'],acceptable,"['CHEMBL635199', 'CHEMBL702884', 'CHEMBL683300...","['CHEMBL312', 'CHEMBL612558', 'CHEMBL2362975',..."
4,Prednisone,https://www.cancer.gov/about-cancer/treatment/...,CHEMBL635,['CHEMBL635'],acceptable,"['CHEMBL632931', 'CHEMBL639855', 'CHEMBL638828...","['CHEMBL612558', 'CHEMBL2362975', 'CHEMBL3253'..."
5,Rituxan (Rituximab),https://www.cancer.gov/about-cancer/treatment/...,CHEMBL1201576,['CHEMBL1201576'],acceptable,"['CHEMBL3137722', 'CHEMBL3137723', 'CHEMBL3137...","['CHEMBL3879801', 'CHEMBL612545']"
6,Venclexta (Venetoclax),https://www.cancer.gov/about-cancer/treatment/...,CHEMBL3137309,['CHEMBL3137309'],acceptable,"['CHEMBL3215001', 'CHEMBL3795379', 'CHEMBL3795...","['CHEMBL3879801', 'CHEMBL4860', 'CHEMBL4625', ..."


In [7]:
def str2list(s: str) -> list:
    """
    Convert a string to a list of strings
    """
    return s.replace("'", "").strip('][').split(', ')

In [8]:
target_dis = new_client.target
all_drug_uni_ids = []

for all_drug_targets in target_df['target_chembl_id']:
    drug_targets_uni_ids = []
    target_ids = str2list(all_drug_targets)
    for target_id in target_ids:
        try:
            first_target_component = target_dis.filter(target_chembl_id=target_id)[0]['target_components'][0]
            target_component_xrefs = first_target_component['target_component_xrefs']
            uniprot_xrefs = [xref for xref in target_component_xrefs if xref['xref_src_db'] == 'UniProt']
            uniprot_ids = [xref['xref_id'] for xref in uniprot_xrefs]
            # Append target ID along with UniProt IDs
            drug_targets_uni_ids.append((target_id, uniprot_ids))
        except IndexError:
            # Handle the case when UniProt IDs are not available
            drug_targets_uni_ids.append((target_id, 'no_uni_id!'))
    # Append drug_targets_uni_ids to all_drug_uni_ids
    all_drug_uni_ids.append(drug_targets_uni_ids)


In [9]:
drug_names = target_df['parent_chembl_id']
dfs = []
for i, drug_data in enumerate(all_drug_uni_ids):
    df = pd.DataFrame(drug_data, columns=[f"{drug_names[i]}_target_id", f"{drug_names[i]}_target_uni_ids"])
    dfs.append(df)
uni_df = pd.concat(dfs, axis=1)
uni_df

,CHEMBL1489_target_id,CHEMBL1489_target_uni_ids,CHEMBL178_target_id,CHEMBL178_target_uni_ids,CHEMBL803_target_id,CHEMBL803_target_uni_ids,CHEMBL384467_target_id,CHEMBL384467_target_uni_ids,CHEMBL635_target_id,CHEMBL635_target_uni_ids,CHEMBL1201576_target_id,CHEMBL1201576_target_uni_ids,CHEMBL3137309_target_id,CHEMBL3137309_target_uni_ids
0,CHEMBL2362975,no_uni_id!,CHEMBL614516,no_uni_id!,CHEMBL613859,no_uni_id!,CHEMBL312,[P12527],CHEMBL612558,no_uni_id!,CHEMBL3879801,no_uni_id!,CHEMBL3879801,no_uni_id!
1,CHEMBL2094118,"[B0LPE2, P08588, Q5T5Y4, Q9UKG7, Q9UKG8]",CHEMBL614514,no_uni_id!,CHEMBL2362975,no_uni_id!,CHEMBL612558,no_uni_id!,CHEMBL2362975,no_uni_id!,CHEMBL612545,no_uni_id!,CHEMBL4860,"[C9JHD5, P10415, P10416, Q13842, Q16197]"
2,CHEMBL386,no_uni_id!,CHEMBL2362975,no_uni_id!,CHEMBL614270,no_uni_id!,CHEMBL2362975,no_uni_id!,CHEMBL3253,"[E7ESS9, O95574, P02768, P04277, Q13140, Q645G...",NaN,NaN,CHEMBL4625,"[E1P5L6, Q07817, Q5CZ89, Q5TE65, Q92976]"
3,CHEMBL384,no_uni_id!,CHEMBL612558,no_uni_id!,CHEMBL381,no_uni_id!,CHEMBL372,no_uni_id!,CHEMBL613294,no_uni_id!,NaN,NaN,CHEMBL4361,"[B2R6B2, D3DV03, D3DV04, Q07820, Q9HD91, Q9NRQ..."
4,CHEMBL389,no_uni_id!,CHEMBL345,no_uni_id!,CHEMBL2094118,"[B0LPE2, P08588, Q5T5Y4, Q9UKG7, Q9UKG8]",CHEMBL2034,"[A0ZXF9, B0LPG8, D3DQF4, F5ATB7, P04150, P0415...",CHEMBL376,no_uni_id!,NaN,NaN,CHEMBL4677,"[A8K0F4, Q2M3U0, Q5U0H4, Q92843]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,NaN,NaN,NaN,NaN,CHEMBL4523507,"[B4E2A5, E1P5V2, Q5JXG8, Q8N491, Q9H004, Q9H00...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
857,NaN,NaN,NaN,NaN,CHEMBL3706572,no_uni_id!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
858,NaN,NaN,NaN,NaN,CHEMBL4523582,[P0DTD1],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
859,NaN,NaN,NaN,NaN,CHEMBL613834,no_uni_id!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
uni_df.to_csv('Accep_target2uni_id_df.csv', index=False)

# Merge dataset

In [11]:
import pandas as pd

accept_df = pd.read_csv('Accep_target2uni_id_df.csv')

chembl_id_columns = [each.split("_")[0] for each in accept_df.columns.to_list()]
chembl_id_columns = list(pd.Series(chembl_id_columns).unique())

df = pd.DataFrame(columns=['parent_compound_id', 'target_id', 'target_uniprot_ids'])
for each_chembl_id in chembl_id_columns:
    temp_df = accept_df[[f"{each_chembl_id}_target_id", f"{each_chembl_id}_target_uni_ids"]]
    temp_df.columns = ['target_id', 'target_uniprot_ids']
    temp_df['parent_compound_id'] = each_chembl_id
    temp_df['compound_specificity'] = "acceptable"

    df = pd.concat([df, temp_df], axis=0)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)


/tmp/ipykernel_94321/2847640752.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['parent_compound_id'] = each_chembl_id
/tmp/ipykernel_94321/2847640752.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['compound_specificity'] = "acceptable"
/tmp/ipykernel_94321/2847640752.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [13]:
specific_df = pd.read_csv('Specific_target2uni_id_df.csv')

chembl_id_columns = [each.split("_")[0] for each in specific_df.columns.to_list()]
chembl_id_columns = list(pd.Series(chembl_id_columns).unique())

for each_chembl_id in chembl_id_columns:
    temp_df = specific_df[[f"{each_chembl_id}_target_id", f"{each_chembl_id}_target_uni_ids"]]
    temp_df.columns = ['target_id', 'target_uniprot_ids']
    temp_df['parent_compound_id'] = each_chembl_id
    temp_df['compound_specificity'] = "specific"

    df = pd.concat([df, temp_df], axis=0)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)



/tmp/ipykernel_94321/875157239.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['parent_compound_id'] = each_chembl_id
/tmp/ipykernel_94321/875157239.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['compound_specificity'] = "specific"
/tmp/ipykernel_94321/875157239.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [14]:
df

,parent_compound_id,target_id,target_uniprot_ids,compound_specificity
0,CHEMBL1489,CHEMBL2362975,no_uni_id!,acceptable
1,CHEMBL1489,CHEMBL2094118,"['B0LPE2', 'P08588', 'Q5T5Y4', 'Q9UKG7', 'Q9UK...",acceptable
2,CHEMBL1489,CHEMBL386,no_uni_id!,acceptable
3,CHEMBL1489,CHEMBL384,no_uni_id!,acceptable
4,CHEMBL1489,CHEMBL389,no_uni_id!,acceptable
...,...,...,...,...
4417,CHEMBL727,CHEMBL4523582,['P0DTD1'],specific
4418,CHEMBL727,CHEMBL612600,no_uni_id!,specific
4419,CHEMBL727,CHEMBL4303840,no_uni_id!,specific
4420,CHEMBL727,CHEMBL3706568,no_uni_id!,specific


In [15]:
df.to_csv('AML_accep_specific_uni_id_df.csv', index=False)